In [1]:
import requests
import json
import pandas as pd
import os
import datetime


url_parliaments = "https://www.abgeordnetenwatch.de/api/v2/parliaments"
url_parliament_periods = "https://www.abgeordnetenwatch.de/api/v2/parliament-periods"
url_politicians = "https://www.abgeordnetenwatch.de/api/v2/politicians"
url_candidacies_mandates = "https://www.abgeordnetenwatch.de/api/v2/candidacies-mandates"


def get_path():
    if os.path.isdir(os.path.join(os.getcwd(), "files")):
        save_path = os.path.join(os.getcwd(), "files")
    else:
        save_path = os.path.join(os.getcwd(), "AW_scraper","files")
    return save_path

def load_data(link, range_ = 50):
    x = 0
    json_list = list()
    if range_ == None:
        url_parliament_periods = link
        myResponse = requests.get(url_parliament_periods)
        print(x)
        jData = json.loads(myResponse.content)
        if x == 0: 
#            print(jData)
            json_list.append(jData)


    else:
        while x >= 0:
            url_parliament_periods = link+f"?page={x}&pager_limit=1000"
            #'f"?id[gte]={x}&id[lt]={x+range_}"

            myResponse = requests.get(url_parliament_periods)
            print(x)
            jData = json.loads(myResponse.content)
            if x == 0: 
    #            print(jData)
                json_list.append(jData)
                x += 1
            else:
                if not json_list[-1]["data"]:
                    z = x
                    x = -1                
                else:
                    x += 1
                    json_list.append(jData)
            if x>=15850:
                x=-1
#        print(f"loaded {50*z} lines")



    return json_list, jData



In [2]:
url_parliament_periods = "https://www.abgeordnetenwatch.de/api/v2/parliament-periods" #?id[gte]={x}&id[lt]={x+50}"
json_list, jData = load_data(url_parliament_periods)


0
1
2


In [3]:
json_list

[{'meta': {'abgeordnetenwatch_api': {'version': '2.7',
    'changelog': 'https://www.abgeordnetenwatch.de/api/version-changelog/aktuell',
    'licence': 'CC0 1.0',
    'licence_link': 'https://creativecommons.org/publicdomain/zero/1.0/deed.de',
    'documentation': 'https://www.abgeordnetenwatch.de/api/entitaeten/parliament-period'},
   'status': 'ok',
   'status_message': '',
   'result': {'count': 140,
    'total': 140,
    'page': '0',
    'results_per_page': 1000}},
  'data': [{'id': 158,
    'entity_type': 'parliament_period',
    'label': 'Brandenburg 2024 - 2029',
    'api_url': 'https://www.abgeordnetenwatch.de/api/v2/parliament-periods/158',
    'abgeordnetenwatch_url': 'https://www.abgeordnetenwatch.de/brandenburg',
    'parliament': {'id': 16,
     'entity_type': 'parliament',
     'label': 'Brandenburg',
     'api_url': 'https://www.abgeordnetenwatch.de/api/v2/parliaments/16',
     'abgeordnetenwatch_url': 'https://www.abgeordnetenwatch.de/brandenburg'},
    'previous_perio

In [ ]:

rows = list()
max_len_fractions=1

for i in json_list:
    candidacies_mandates_df = pd.DataFrame()
    counter = 0
    for key, value in i.items():
        if key == "data":
            for element in value:
                current_element=element.copy()
                if counter == 0:
                    print(element)
                    first_element = element.copy()

                if 'fraction_membership' in current_element:
                    fraction_list = [current_element['fraction_membership'][u]['label'] for u,v, in enumerate(current_element['fraction_membership']) ]
                    if len(fraction_list) > max_len_fractions:
                        max_len_fractions = len(fraction_list)
                    fraction_list = fraction_list + [None for z in range(max_len_fractions-len(fraction_list))]
                    fraction_dict = {"fraction_membership"+str(l+1):v for l,v in enumerate(fraction_list)}
                    current_element.pop('fraction_membership')
                else:
                    fraction_dict = {"fraction_membership"+str(u+1):None for u in range(max_len_fractions)}

                if current_element['parliament_period']['label'] != None:
                    parliamend_period_dict = {"parliament_period": current_element['parliament_period']["label"]}
                else:
                    parliamend_period_dict = {"parliamend_period": None}
                current_element.pop('parliament_period')

                if current_element['electoral_data'] != None:
                    if current_element['electoral_data']['electoral_list'] != None:
                        electoral_list_dict = {'electoral_list':current_element['electoral_data']['electoral_list']['label']}
                    else:
                        electoral_list_dict = {'electoral_list':None}
                    if current_element['electoral_data']['constituency'] != None:
                        constituency_dict = {'constituency':current_element['electoral_data']['constituency']['label']}
                    else:
                        constituency_dict = {'constituency':None}
                current_element.pop('electoral_data')

                if current_element["politician"] != None:
                    # current_element = current_element | {'politician': current_element['politician']['label'] }
                    politician_dict =  {'politician': current_element['politician']['label'] }
                else:
                    politician_dict =  {'politician': None}
                current_element.pop('politician')

                rows.append(current_element | fraction_dict | politician_dict | electoral_list_dict | constituency_dict | parliamend_period_dict | fraction_dict)

                counter += 1

save_path = "files"
#final_df.to_excel(os.path.join(save_path,"candidacies_mandate.xlsx"))
print(f"candidacies mandates end: {datetime.datetime.now()} \n")
print(f"max_len_fractions: {max_len_fractions}")
candidacies_mandates_df = pd.DataFrame(rows)
candidacies_mandates_df.to_excel("files\\candidacies_mandates.xlsx", index=False)

In [4]:
df_list = list()
rows = list()
for i in json_list:
    parliaments_periods_df = pd.DataFrame()
    counter = 0
    for key, value in i.items():
        if key == "data":
            for element in value:
                current_element=element.copy()
                if counter == 0:
                    print(element)
                    first_element = element.copy()

                if 'parliament' in current_element:
                    parliament_dict = {"parliament": current_element['label']}
                counter += 1
                rows.append(current_element | parliament_dict)




{'id': 158, 'entity_type': 'parliament_period', 'label': 'Brandenburg 2024 - 2029', 'api_url': 'https://www.abgeordnetenwatch.de/api/v2/parliament-periods/158', 'abgeordnetenwatch_url': 'https://www.abgeordnetenwatch.de/brandenburg', 'parliament': {'id': 16, 'entity_type': 'parliament', 'label': 'Brandenburg', 'api_url': 'https://www.abgeordnetenwatch.de/api/v2/parliaments/16', 'abgeordnetenwatch_url': 'https://www.abgeordnetenwatch.de/brandenburg'}, 'previous_period': {'id': 154, 'entity_type': 'parliament_period', 'label': 'Brandenburg Wahl 2024', 'api_url': 'https://www.abgeordnetenwatch.de/api/v2/parliament-periods/154', 'abgeordnetenwatch_url': 'https://www.abgeordnetenwatch.de/brandenburg/wahl-2024'}, 'type': 'legislature', 'election_date': None, 'start_date_period': '2024-10-17', 'end_date_period': '2029-10-22'}


In [5]:
parliament_periods_df = pd.DataFrame(rows)


In [18]:
parliament_periods_df = pd.DataFrame(rows)
parliament_periods_df.to_excel("files\\parliament_periods.xlsx")

In [17]:

parliament_periods_df.head()

,id,entity_type,label,api_url,abgeordnetenwatch_url,parliament,previous_period,type,election_date,start_date_period,end_date_period
0,158,parliament_period,Brandenburg 2024 - 2029,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/brandenburg,Brandenburg 2024 - 2029,"{'id': 154, 'entity_type': 'parliament_period'...",legislature,None,2024-10-17,2029-10-22
1,157,parliament_period,Sachsen 2024 - 2029,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/sachsen,Sachsen 2024 - 2029,"{'id': 153, 'entity_type': 'parliament_period'...",legislature,None,2024-10-01,2029-09-30
2,156,parliament_period,Thüringen 2024 - 2029,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/thueringen,Thüringen 2024 - 2029,"{'id': 152, 'entity_type': 'parliament_period'...",legislature,None,2024-09-26,2029-09-30
3,155,parliament_period,EU-Parlament 2024 - 2029,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/eu,EU-Parlament 2024 - 2029,"{'id': 151, 'entity_type': 'parliament_period'...",legislature,None,2024-07-16,2029-07-15
4,154,parliament_period,Brandenburg Wahl 2024,https://www.abgeordnetenwatch.de/api/v2/parlia...,https://www.abgeordnetenwatch.de/brandenburg/w...,Brandenburg Wahl 2024,None,election,2024-09-22,2024-08-15,2024-09-30


In [ ]:
final_df = pd.concat(df_list)
final_df.to_excel(os.path.join(save_path,"parliament_period.xlsx"), index=False)
print(f"parliament periods end: {datetime.datetime.now()}\n")
